In [1]:
using CSV
using JSON
using Glob

using FilePaths; using FilePathsBase: /

using DataFrames

In [10]:
SOURCE::PosixPath     = Path("/mnt/ibm_lg/merscope/merfish_raw_data/202205191322_MsKidney-wtK3-VZG116b-VT-061922_VMSC02401");
CODEBOOK::PosixPath   = glob("codebook*.csv", SOURCE)[1];
DATA_CSV::PosixPath   = SOURCE/Path("dataorganization.csv");

OUT::PosixPath        = Path("/mnt/ibm_lg/jai.hebel/starfish-data")/SOURCE[end];

mkpath(OUT/"data");

In [12]:
df = DataFrame(CSV.File(CODEBOOK))

┌ Warning: User 11005299 not found.
└ @ FilePathsBase /home/jai.hebel/.julia/packages/FilePathsBase/9kSEl/src/libc.jl:86
┌ Warning: Group 11000513 not found.
└ @ FilePathsBase /home/jai.hebel/.julia/packages/FilePathsBase/9kSEl/src/libc.jl:127


Row,name,id,barcodeType,V0001T8B1,V0002T8B1,V0003T8B1,V0004T8B1,V0005T8B1,V0006T8B1,V0007T8B1,V0008T8B1,V0009T8B1,V0010T8B1,V0011T8B1,V0012T8B1,V0013T8B1,V0014T8B1,V0015T8B1,V0016T8B1,V0017T8B1,V0018T8B1,V0019T8B1,V0154T8B1,V0021T8B1
,String15,String31,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,Sox17,ENSMUST00000027035,merfish,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,Zap70,ENSMUST00000027291,merfish,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0
3,Slc40a1,ENSMUST00000027137,merfish,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,Cxcr2,ENSMUST00000027372,merfish,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0
5,Des,ENSMUST00000027409,merfish,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0
6,Scg2,ENSMUST00000185234,merfish,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
7,Marco,ENSMUST00000027639,merfish,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
8,Fcmr,ENSMUST00000038829,merfish,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
9,Prelp,ENSMUST00000048432,merfish,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
def load_csv_codebook(csv_path):
   """ CSV to JSON codebook
   """
   csv_codebook: pd.DataFrame = pd.read_csv(csv_path, index_col=0)
   csv_codebook = csv_codebook.drop(columns=["id"]) # Drop ID column

   N_CHANNELS = 3 # Assuming 3 channels
   RC_MAP = { # Each bitcode name corresponds to a (round, channel) combination
   csv_codebook.columns[i]: (i//N_CHANNELS, i%N_CHANNELS) 
   for i in range(csv_codebook.shape[1])
   }

   def parse_codewords(row):
      row = row[row == 1]
      codewords = [
         {
            Axes.ROUND.value:       r,
            Axes.CH.value:          c,
            Features.CODE_VALUE:    1
         } for (r,c) in (RC_MAP[marker] for marker in row.index)
      ]
      return {
         Features.CODEWORD:      codewords,
         Features.TARGET:        row.name
      }
      

   mappings = csv_codebook.apply(parse_codewords, axis=1).to_list()
   return Codebook.from_code_array(mappings)